# STEP 3: preprocess_get_behaviors

Press SHIFT + ENTER to run code

### USER INPUT!

In [1]:
## Define project
project_name = 'project_acc_MiniscopeCap'

### MAIN CODE: predict behaviors based on features
USER INPUT NEEDED BELOW!!! 
WAIT UNTIL CODE FINISHES PROCESSING DATA

In [2]:
###### RUN DEPENDENCIES ######
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from matplotlib.colors import ListedColormap
import os
import sys
if not os.path.join(os.path.abspath(''), '../') in sys.path:
    sys.path.append(os.path.join(os.path.abspath(''), '../'))
from utils.classification import load_model, load_features, load_data, weighted_smoothing, load_behaviors
from utils.meta import *
from tqdm import notebook
import pickle
import os
import pandas as pd


###### LOAD: model, preprocessed pose, and binned features ######
model_path = '../model/model.pkl'
model = load_model(model_path)
data_path = f"../processed_dataset/{project_name}/raw_data_{project_name}.pkl"
poses = load_data(data_path)
features_path = f"../processed_dataset/{project_name}/binned_features_{project_name}.pkl"
features = load_features(features_path)

repeat_n = 6
if f"behaviors_{project_name}.pkl" not in os.listdir(f"../processed_dataset/{project_name}"):
    behaviors = {key: [] for key in groups_project_acc_MiniscopeCap} # *** CHANGE to groups_{project_name} manually *** #
    for group in notebook.tqdm(groups_project_acc_MiniscopeCap): # *** CHANGE to groups_{project_name} manually *** #
        behaviors[group] = {key: [] for key in conditions_project_acc_MiniscopeCap} # *** CHANGE to conditions_{project_name} manually *** #
        for condition in notebook.tqdm(conditions_project_acc_MiniscopeCap): # *** CHANGE to conditions_{project_name} manually *** #
            behaviors[group][condition] = {key: [] for key in [f'file{i}' 
                                               for i in range(len(poses[group][condition]))]}
    
            for i in range(len(poses[group][condition])):
                total_n_frames = poses[group][condition][f'file{i}'].shape[0]
                # predict downsampled features
                predict_ds = model.predict(features[group][condition][f'file{i}'][0])
                # upsample by duplicating
                predictions = np.pad(predict_ds.repeat(repeat_n), (repeat_n, 0), 'edge')[:total_n_frames]
                # smooth with minimum 12 frames (200ms)
                behaviors[group][condition][f'file{i}'] = weighted_smoothing(predictions, size=12)
else:
    behaviors = load_behaviors(f"../processed_dataset/{project_name}/behaviors_{project_name}.pkl")

###### WAIT UNTIL CODE FINISHES PROCESSING DATA ######

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

In [3]:
###### STORE ALL BEHAVIORS IN DICTIONARY ######  
if f"behaviors_{project_name}.pkl" not in os.listdir(f"../processed_dataset/{project_name}"):
    with open(f"../processed_dataset/{project_name}/behaviors_{project_name}.pkl", 'wb') as f:
        pickle.dump(behaviors, f)
else:
    print('Predicted behaviors exists!')

# PREPROCESS COMPLETE! Move on to behavior analysis + figure generation!